In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle as pkl
import os
from SamBA.samba import NeighborHoodClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from random_scm.random_scm_classifier import RandomScmClassifier
from summit.multiview_platform.monoview_classifiers.scm import SCM
from sklearn.ensemble import RandomForestClassifier

rs = np.random.RandomState(42)

splits = 100

def best_feats(importances, limit=5):
    best_inds = np.argsort(-importances)
    return dict((str(ind), np.round(importances[ind], 2)) for ind in best_inds[:limit])

def benchmark(X, y, n_splits=10, train_size=0.6):
    neigh_clf = NeighborHoodClassifier(
                     base_estimator=DecisionTreeClassifier(max_depth=1,
                                                           splitter='best',
                                                           criterion='gini'),
                     n_estimators=10,
                    vote_compensate=False)
    rscm_clf = RandomScmClassifier(n_estimators=10, max_rules=10,
                     p_options=[1.0],
                     model_type="conjunction",
                     random_state=rs)
    scm_clf = SCM(
                random_state=rs,
                model_type="conjunction",
                max_rules=10,
                p=1.0)
    rf_clf = RandomForestClassifier(n_estimators=10, random_state=rs, max_depth=3)

    accuracies = np.zeros((n_splits, 4))
    n_feats = np.zeros((n_splits, 4))
    feature_importances = np.zeros((X.shape[1], n_splits, 4))
    for i in range(n_splits):
        X_train, X_test, y_train, y_test = train_test_split(X, y, shuffle=True, 
                                                            random_state=rs, train_size=train_size, stratify=y)
        neigh_clf.fit(X_train, y_train)
        neigh_clf_preds = neigh_clf.predict(X_test)
        accuracies[i, 0] = accuracy_score(neigh_clf_preds, y_test)
        feature_importances[:, i, 0] = neigh_clf.feature_importances_
        n_feats[i, 0]= len(np.where(neigh_clf.feature_importances_!=0)[0])
        
        try:
            rscm_clf.fit(X_train, y_train)
            rscm_clf_preds = rscm_clf.predict(X_test)
            accuracies[i, 1] = accuracy_score(rscm_clf_preds, y_test)
            feature_importances[:, i, 1] = rscm_clf.feature_importances_
            n_feats[i, 1]= len(np.where(rscm_clf.feature_importances_!=0)[0])
        except ValueError:
            accuracies[i, 1] = 0
            n_feats[i, 1] = 0
            feature_importances[:, i, 1] = np.zeros(X.shape[1])
            
        scm_clf.fit(X_train, y_train)
        scm_clf_preds = scm_clf.predict(X_test)
        accuracies[i, 2] = accuracy_score(scm_clf_preds, y_test)
        feature_importances[:, i, 2] = scm_clf.feature_importances_
        n_feats[i, 2]= len(np.where(scm_clf.feature_importances_!=0)[0])
        
        rf_clf.fit(X_train, y_train)
        rf_clf_preds = rf_clf.predict(X_test)
        accuracies[i, 3] = accuracy_score(rf_clf_preds, y_test)        
        feature_importances[:, i, 3] = rf_clf.feature_importances_       
        n_feats[i, 3]= len(np.where(rf_clf.feature_importances_!=0)[0])

    mean_accs = np.round(np.mean(accuracies, axis=0), 2)
    stds_accs = np.round(np.std(accuracies, axis=0), 2)
    mean_n_feats = np.round(np.mean(n_feats, axis=0), 2)
    mean_feature_importances = np.round(np.mean(feature_importances, axis=1),2)
    
    print("NeighborHoodClassifier has a mean accuracy of \n\t{} +/-{}, relying on \n\t{} features \n\t {} ".format(mean_accs[0], stds_accs[0], mean_n_feats[0], best_feats(mean_feature_importances[:,0])))
    print("RandomSCM has a mean accuracy of \n\t{} +/-{}, relying on \n\t{} features \n\t {} ".format(mean_accs[1], stds_accs[1], mean_n_feats[1], best_feats(mean_feature_importances[:,1])))
    print("VanillaSCM has a mean accuracy of \n\t{} +/-{}, relying on \n\t{} features \n\t {} ".format(mean_accs[2], stds_accs[2], mean_n_feats[2], best_feats(mean_feature_importances[:,2])))
    print("RandomForest has a mean accuracy of \n\t{} +/-{}, relying on \n\t{} features \n\t {} ".format(mean_accs[3], stds_accs[3], mean_n_feats[3], best_feats(mean_feature_importances[:,3])))
    
    return accuracies, n_feats

# Biological experiments with Neighborhood classifier

## Recover dataset

Task : predict the persitency of covid symptoms with proteomics, and metabolomics.
To extract the data, we use a script, slightly modified. 

First, let's extract the metadata that will provide the classification labels :

In [2]:
data_path = '/home/baptiste/Documents/Datasets/recover'

metadata_filename = os.path.join(data_path, 'metadata.csv')
meta_df = pd.read_csv(metadata_filename)
meta_df.columns = ['#', 'plate', '-', 'symptoms'] + list(meta_df)[4:]
print('labels metadata :', meta_df['symptoms'].to_list())
meta_idx = meta_df['ID'].to_list()
meta_label = meta_df['symptoms'].to_list()
meta_id_label_dict = {str(k): 1 if v=='S' else 0 for k, v in zip(meta_idx, meta_label)}

labels metadata : ['S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'S', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS', 'NS']


Then, we extract the first proteomic features

In [3]:
proteomics_data_filename = os.path.join(data_path, 'proteomics.csv')

dim_df = pd.read_csv(proteomics_data_filename, nrows=1)
dim = len(list(dim_df))
all_cols = [i for i in range(dim)]
feat_cols = all_cols[1:-4]
samplesidx_col = [0]

feat_df = pd.read_csv(proteomics_data_filename, skiprows=4, nrows=1, dtype=str, usecols=feat_cols)
features = list(feat_df)

idx_df = pd.read_csv(proteomics_data_filename, skiprows=6, index_col=0, skipfooter=4, usecols=[0], engine='python')
idx = list(idx_df.index.values)

df1 = pd.read_csv(proteomics_data_filename, skiprows=6, dtype=np.float32, skipfooter=4, usecols=feat_cols, engine='python')
assert df1.shape[0] == len(idx)
assert df1.shape[1] == len(features)

df1['idx'] = idx
df1.set_index('idx', inplace=True)
df1.columns = features

#clean data of samples that are not in metadata :
idx = df1.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df1['label'] = y
df1 = df1[df1.label != 'to_remove']

#create X and y matrices for ML :
y = list(df1['label'])
del df1['label']
X = df1.to_numpy()
print('proteomics data :')
print('# of samples : ', df1.shape[0])
print('# of features : ', df1.shape[1])
print('labels:', list(dict.fromkeys(y)))

proteomics data :
# of samples :  100
# of features :  184
labels: [1, 0]


Let's run the neighborhood classifier on this data with several train/test splits : 

In [4]:
accuracies_prot_1, n_feats_prot_1 = benchmark(X, y, n_splits=splits)

NeighborHoodClassifier has a mean accuracy of 
	0.5 +/-0.06, relying on 
	2.18 features 
	 {'86': 0.06, '122': 0.06, '84': 0.06, '39': 0.05, '37': 0.04} 
RandomSCM has a mean accuracy of 
	0.52 +/-0.06, relying on 
	21.53 features 
	 {'86': 0.04, '37': 0.03, '39': 0.03, '67': 0.03, '53': 0.02} 
VanillaSCM has a mean accuracy of 
	0.51 +/-0.07, relying on 
	3.51 features 
	 {'86': 0.06, '5': 0.05, '0': 0.04, '17': 0.03, '37': 0.03} 
RandomForest has a mean accuracy of 
	0.53 +/-0.07, relying on 
	43.16 features 
	 {'67': 0.02, '37': 0.02, '39': 0.02, '0': 0.01, '108': 0.01} 


Let's chek the second proteomic group of features 

In [5]:
print('\nPROTEOMICS CYTOKINES DATA :')
proteomics_cyt_data_filename = os.path.join(data_path, 'proteomics_cyt.csv')

dim_df = pd.read_csv(proteomics_cyt_data_filename, nrows=1)
dim = len(list(dim_df))
all_cols = [i for i in range(dim)]
feat_cols = all_cols[1:-2]
samplesidx_col = [0]

feat_df = pd.read_csv(proteomics_cyt_data_filename, skiprows=4, nrows=1, dtype=str, usecols=feat_cols)
features = list(feat_df)

idx_df = pd.read_csv(proteomics_cyt_data_filename, skiprows=7, index_col=0, skipfooter=14, usecols=[0], engine='python')
idx = list(idx_df.index.values)

df2 = pd.read_csv(proteomics_cyt_data_filename, skiprows=7, dtype=np.float32, skipfooter=14, usecols=feat_cols, na_values=['> ULOQ'], engine='python')
assert df2.shape[0] == len(idx)
assert df2.shape[1] == len(features)

df2['idx'] = idx
df2.set_index('idx', inplace=True)
df2.columns = features

#clean data of samples that are not in metadata :
idx = df2.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df2['label'] = y
df2 = df2[df2.label != 'to_remove']

for col in list(df2):
   df2[col].fillna(int(df2[col].mean()), inplace=True)


#create X and y matrices for ML :
y = list(df2['label'])
del df2['label']
X = df2.to_numpy()
print('# of samples : ', df2.shape[0])
print('# of features : ', df2.shape[1])



PROTEOMICS CYTOKINES DATA :
# of samples :  100
# of features :  45


In [6]:
accuracies_prot_2, n_feats_prot_2 = benchmark(X, y, n_splits=splits)

/home/baptiste/Documents/venvs/develop/lib/python3.6/site-packages/samba-0.0.dev0-py3.6.egg/SamBA/distances.py:35: RuntimeWarning:

overflow encountered in exp

/home/baptiste/Documents/venvs/develop/lib/python3.6/site-packages/samba-0.0.dev0-py3.6.egg/SamBA/samba.py:164: RuntimeWarning:

divide by zero encountered in true_divide

/home/baptiste/Documents/venvs/develop/lib/python3.6/site-packages/numpy/core/fromnumeric.py:90: RuntimeWarning:

invalid value encountered in reduce



ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

Now with the meatbolomics : 

In [ ]:
print('\nMETABOLOMICS DATA :')
metabolomics_data_filename = os.path.join(data_path, 'metabolomics.csv')
feat_df = pd.read_csv(metabolomics_data_filename, index_col=0, skiprows=[0], dtype=str, usecols=[0])
features = list(feat_df.index.values)

idx_df = pd.read_csv(metabolomics_data_filename, header=1, nrows=1)
idx = list(idx_df)[1:]
idx = [l[17:22] for l in idx]

labels_df = pd.read_csv(metabolomics_data_filename, nrows=1)
labels = list(labels_df)[1:]

cols_df = pd.read_csv(metabolomics_data_filename, header=1, nrows=1)
cols_list = list(cols_df)

df3 = pd.read_csv(metabolomics_data_filename, header=1, dtype=np.float32, na_values=['#DIV/0!'], usecols=cols_list[1:])
df3 = df3.T
df3['idx'] = idx
df3.set_index('idx', inplace=True)
df3.columns = features
df3 = df3.dropna(axis=1)

#clean data of samples that are not in metadata :
idx = df3.index.values
y = []
for k in range(len(idx)):
    id = idx[k]
    if id in meta_id_label_dict:
        y.append(meta_id_label_dict[id])
    else:
        # we will not put this sample in the dataset
        #print('sample to remove because of unknown label:', k, id)
        y.append('to_remove')
df3['label'] = y
df3 = df3[df3.label != 'to_remove']

#create X and y matrices for ML :
y = list(df3['label'])
del df3['label']
X = df3.to_numpy()

print('metabolomics data :')
print('# of samples : ', df3.shape[0])
print('# of features : ', df3.shape[1])


In [ ]:
accuracies_meta, n_feats_meta = benchmark(X, y, n_splits=splits)

Now if we try to aggregate some of the views : 

In [ ]:

df_1_2 = pd.concat([df1, df2], axis=1)
X = df_1_2.to_numpy()
print(df_1_2.columns[189], df_1_2.columns[190])
print('Full proteomic data :')
print('# of samples : ', df_1_2.shape[0])
print('# of features : ', df_1_2.shape[1])

accuracies_prot_f, n_feats_prot_f = benchmark(X, y, n_splits=splits)

Now on the three types of data :

In [ ]:
df = pd.concat([df_1_2, df3], axis=1)
df = df.dropna(axis=0)
print('Multi-omics df :')
print('# of samples : ', df.shape[0])
print('# of features : ', df.shape[1])
X = df.to_numpy()

accuracies_f, n_feats_f = benchmark(X, y, n_splits=splits)

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Patch
from matplotlib.lines import Line2D

plt_1 = plt.figure(figsize=(15, 15))

legend_elements = [Line2D([], [], marker='o', color='k', label='Metabo',markersize=10, linestyle='None'),
                   Line2D([], [], marker='^', color='k', label='Multiomic',markersize=10, linestyle='None'),
                   Line2D([], [], marker='s', color='k', label='Proteo full',markersize=10, linestyle='None'),
                   Line2D([], [], marker='*', color='k', label='Proteo1',markersize=10, linestyle='None'),
                   Line2D([], [], marker='D', color='k', label='Proteo2',markersize=10, linestyle='None'),
                   Patch(facecolor='b', label='R-SCM'),
                   Patch(facecolor='g', label='SCM'),
                   Patch(facecolor='r', label='RF')]
markersize=20
alpha=0.5
plt.scatter(accuracies_f[:,0], accuracies_f[:,1], marker='^', color="b", s=markersize, alpha=alpha)
plt.scatter(accuracies_f[:,0], accuracies_f[:,2], marker='^', color="g", s=markersize, alpha=alpha)
plt.scatter(accuracies_f[:,0], accuracies_f[:,3], marker='^', color="r", s=markersize, alpha=alpha)
plt.scatter(accuracies_meta[:,0], accuracies_meta[:,1], marker='o', color="b", s=markersize, alpha=alpha)
plt.scatter(accuracies_meta[:,0], accuracies_meta[:,2], marker='o', color="g", s=markersize, alpha=alpha)
plt.scatter(accuracies_meta[:,0], accuracies_meta[:,3], marker='o', color="r", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_f[:,0], accuracies_prot_f[:,1], marker='s', color="b", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_f[:,0], accuracies_prot_f[:,2], marker='s', color="g", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_f[:,0], accuracies_prot_f[:,3], marker='s', color="r", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_1[:,0], accuracies_prot_1[:,1], marker='*', color="b", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_1[:,0], accuracies_prot_1[:,2], marker='*', color="g", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_1[:,0], accuracies_prot_1[:,3], marker='*', color="r", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_2[:,0], accuracies_prot_2[:,1], marker='D', color="b", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_2[:,0], accuracies_prot_2[:,2], marker='D', color="g", s=markersize, alpha=alpha)
plt.scatter(accuracies_prot_2[:,0], accuracies_prot_2[:,3], marker='D', color="r", s=markersize, alpha=alpha)

# Show the boundary between the regions:
theta = np.arange(0, 1, 0.01)
plt.plot(theta, theta)
plt.ylabel("Other algorithms accuracies")
plt.xlabel("Neighborhood classifier accuracies")
plt.legend(handles=legend_elements)
plt.show()

In [ ]:
plt_1 = plt.figure(figsize=(15, 15))

legend_elements = [Line2D([], [], marker='o', color='k', label='Metabo',markersize=10, linestyle='None'),
                   Line2D([], [], marker='^', color='k', label='Multiomic',markersize=10, linestyle='None'),
                   Line2D([], [], marker='s', color='k', label='Proteo full',markersize=10, linestyle='None'),
                   Line2D([], [], marker='*', color='k', label='Proteo1',markersize=10, linestyle='None'),
                   Line2D([], [], marker='D', color='k', label='Proteo2',markersize=10, linestyle='None'),
                   Patch(facecolor='b', label='R-SCM'),
                   Patch(facecolor='g', label='SCM'),
                   Patch(facecolor='r', label='RF')]
markersize=20
alpha=0.5
plt.scatter(n_feats_f[:,0], n_feats_f[:,1], marker='^', color="b", s=markersize, alpha=alpha)
plt.scatter(n_feats_f[:,0], n_feats_f[:,2], marker='^', color="g", s=markersize, alpha=alpha)
plt.scatter(n_feats_f[:,0], n_feats_f[:,3], marker='^', color="r", s=markersize, alpha=alpha)
plt.scatter(n_feats_meta[:,0], n_feats_meta[:,1], marker='o', color="b", s=markersize, alpha=alpha)
plt.scatter(n_feats_meta[:,0], n_feats_meta[:,2], marker='o', color="g", s=markersize, alpha=alpha)
plt.scatter(n_feats_meta[:,0], n_feats_meta[:,3], marker='o', color="r", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_f[:,0], n_feats_prot_f[:,1], marker='s', color="b", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_f[:,0], n_feats_prot_f[:,2], marker='s', color="g", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_f[:,0], n_feats_prot_f[:,3], marker='s', color="r", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_1[:,0], n_feats_prot_1[:,1], marker='*', color="b", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_1[:,0], n_feats_prot_1[:,2], marker='*', color="g", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_1[:,0], n_feats_prot_1[:,3], marker='*', color="r", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_2[:,0], n_feats_prot_2[:,1], marker='D', color="b", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_2[:,0], n_feats_prot_2[:,2], marker='D', color="g", s=markersize, alpha=alpha)
plt.scatter(n_feats_prot_2[:,0], n_feats_prot_2[:,3], marker='D', color="r", s=markersize, alpha=alpha)

# Show the boundary between the regions:
theta = np.arange(0, 50, 0.01)
plt.plot(theta, theta)
plt.ylabel("Other algorithms n_feat")
plt.xlabel("Neighborhood classifier n_feat")
plt.legend(handles=legend_elements)
plt.show()

In [ ]:
plt_1 = plt.figure(figsize=(15, 15))

markersize=50
alpha=0.7
plt.scatter(X[:,1078], X[:,1103], marker='^', c=y, s=markersize, alpha=alpha, cmap="bwr")
plt.ylabel("7.47_525.8156n")
plt.xlabel("3.93_182.1901m/z")
plt.show()

In [ ]:
plt_1 = plt.figure(figsize=(15, 15))
print(df.columns[3100], df.columns[1245])
markersize=50
alpha=0.7
plt.scatter(X[:,1078], X[:,3100], marker='^', c=y, s=markersize, alpha=alpha, cmap="bwr")
plt.ylabel("9.25_382.3062n")
plt.xlabel("3.93_182.1901m/z")
plt.show()

In [ ]:
plt_1 = plt.figure(figsize=(15, 15))

markersize=50
alpha=0.7
plt.scatter(X[:,1078], X[:,1245], marker='^', c=y, s=markersize, alpha=alpha, cmap="bwr")
plt.ylabel("6.55_561.8297n")
plt.xlabel("3.93_182.1901m/z")
plt.show()

In [ ]:
plt_1 = plt.figure(figsize=(15, 15))

df_1_2 = pd.concat([df1, df2], axis=1)
X = df_1_2.to_numpy()

markersize=50
alpha=0.7
plt.scatter(X[:,189], X[:,211], marker='^', c=y, s=markersize, alpha=alpha, cmap="bwr")
plt.ylabel("Proteo 211")
plt.xlabel("IL2")
plt.show()